# <a id='head'>Содержание проекта</a>
<a href='#01'>Описание проекта</a>  
<a href='#02'>import</a>
1. <a href='#1'>Загрузка и подготовка данных</a>
2. <a href='#2'>Обучение и проверка модели</a>
3. <a href='#3'>Подготовка к расчёту прибыли</a>
4. <a href='#4'>Расчёт прибыли и рисков </a>
5. <a href='#5'>Bootstrap</a>   

<a href='#00'>Вывод</a>

# <a id='01'>Описание проекта</a> 
<a href='#head'>[head]</a>

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль.

Описание данных
id — уникальный идентификатор скважины;
f0, f1, f2 — три признака точек;
product — объём запасов в скважине (тыс. баррелей).

Условия задачи:
При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.
При цене один баррель сырья - 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
Необходимо оставить лишь те регионы, в которых вероятность убытков меньше 2.5%.

# <a id='02'>import</a> 
<a href='#head'>[head]</a>

In [404]:
import pandas as pd
import numpy as np
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# <a id='1'>1. Загрузка и подготовка данных</a> 
<a href='#head'>[head]</a>

In [405]:
geo_0 = pd.read_csv('/datasets/geo_data_0.csv')
display(geo_0.head())
print()
print(geo_0.info())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


In [406]:
geo_1 = pd.read_csv('/datasets/geo_data_1.csv')
display(geo_1.head())
print()
print(geo_1.info())

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


In [407]:
geo_2 = pd.read_csv('/datasets/geo_data_2.csv')
display(geo_2.head())
print()
print(geo_2.info())

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


Пропуски в данных отсутствуют, тип соответствуют данным.
Приводить численные признаки ('f0', 'f1', 'f2') к одному масштабу смысла нет, так как они, судя по всему и есть одного масштаба.

Разделим данные на тестовую и валидационную части для каждого региона. А также уберем столбец "id", так как он не участвует в обучении модели.

In [408]:
def data_split(data):
    data = data.drop('id', axis=1)
    target = data['product']
    features = data.drop('product', axis=1)

    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=42)
    return features_train, features_valid, target_train, target_valid

features_train_geo_0, features_valid_geo_0, target_train_geo_0, target_valid_geo_0 = data_split(geo_0)
features_train_geo_1, features_valid_geo_1, target_train_geo_1, target_valid_geo_1 = data_split(geo_1)
features_train_geo_2, features_valid_geo_2, target_train_geo_2, target_valid_geo_2 = data_split(geo_2)

# <a id='2'>2. Обучение и проверка модели</a> 
<a href='#head'>[head]</a>

In [409]:
def model_linear(features_train, features_valid, target_train, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    predictions = pd.Series(predictions)
    predictions.index = features_valid.index
    answers = target_valid
    mse = mean_squared_error(answers, predictions)
    rmse = mse ** 0.5
    
    return predictions, rmse

predictions_geo_0, rmse_geo_0 = model_linear(features_train_geo_0, features_valid_geo_0, 
                                               target_train_geo_0, target_valid_geo_0)
predictions_geo_1, rmse_geo_1 = model_linear(features_train_geo_1, features_valid_geo_1, 
                                               target_train_geo_1, target_valid_geo_1)
predictions_geo_2, rmse_geo_2 = model_linear(features_train_geo_2, features_valid_geo_2, 
                                               target_train_geo_2, target_valid_geo_2)

print("Средний запас сырья 'geo_0': {:.2f} | RMSE_geo_0: {:.2f}".format(predictions_geo_0.mean(), rmse_geo_0))
print("Средний запас сырья 'geo_1': {:.2f} | RMSE_geo_1: {:.2f}".format(predictions_geo_1.mean(), rmse_geo_1))
print("Средний запас сырья 'geo_2': {:.2f} | RMSE_geo_2: {:.2f}".format(predictions_geo_2.mean(), rmse_geo_2))

Средний запас сырья 'geo_0': 92.40 | RMSE_geo_0: 37.76
Средний запас сырья 'geo_1': 68.71 | RMSE_geo_1: 0.89
Средний запас сырья 'geo_2': 94.77 | RMSE_geo_2: 40.15


Наиболее большие запасы предсказанного сырья у нас во "2" регионе, а наименьшие в "1".
Однако среднеквадратичная ошибка меньше всего в "1" и больше всего во "2" регионе.
Возможно во "2" и "0" регионе есть "чемпионы", которые тянут их вверх, в то время как "1" регион будет примерно одинаков во всех случаях и соответственно ответы с предсказаниями будут ближе друг к другу.

# <a id='3'>3. Подготовка к расчёту прибыли</a> 
<a href='#head'>[head]</a>

In [410]:
BUDGET = 10000000000
PROFIT_PER_PRODUCT = 450000
COUNT_OF_POINTS = 200
BASIC_PRODUCT = BUDGET / PROFIT_PER_PRODUCT
PRODUCT_PER_POINT = BASIC_PRODUCT / COUNT_OF_POINTS
print("Необходимое количество сырья на месторождении для безубыточной добычи:", PRODUCT_PER_POINT)

Необходимое количество сырья на месторождении для безубыточной добычи: 111.11111111111111


Выходит, что во всех регионах в среднем месторождение превышает необходимый для безубыточности запас.

# <a id='4'>4. Расчёт прибыли и рисков </a> 
<a href='#head'>[head]</a>

In [411]:
def profit(data, target):
    data = data.sort_values(ascending=False)
    top200_profit = target[data.index][:COUNT_OF_POINTS]
    profit = (top200_profit.sum() * PROFIT_PER_PRODUCT) - BUDGET
    return profit

profit_geo_0 = profit(predictions_geo_0, target_valid_geo_0)
print("Прибыль региона '0' при максимальных показателях:", profit_geo_0)
profit_geo_1 = profit(predictions_geo_1, target_valid_geo_1)
print("Прибыль региона '1' при максимальных показателях:", profit_geo_1)
profit_geo_2 = profit(predictions_geo_2, target_valid_geo_2)
print("Прибыль региона '2' при максимальных показателях:", profit_geo_2)

Прибыль региона '0' при максимальных показателях: 3359141114.462179
Прибыль региона '1' при максимальных показателях: 2415086696.681511
Прибыль региона '2' при максимальных показателях: 2598571759.374111


# <a id='5'>5. Bootstrap </a> 
<a href='#head'>[head]</a>

In [412]:
def bootstrap(target, probs):
    state = np.random.RandomState(42)
    values = []
    for i in range(1000):
        target_subsample = target.sample(n=500, replace = True, random_state = state)
        probs_subsample = probs[target_subsample.index]
    
        values.append(profit(probs_subsample, target_subsample))

    values = pd.Series(values)
    lower = values.quantile(0.025) 
    upper = values.quantile(0.975) 
    mean = values.mean()
    
    count = 0
    for i in values:
        if i < 0:
            count += 1
    risk = count / 1000 
    
    return risk, lower, upper, mean

risk_geo_0, lower_geo_0, upper_geo_0, mean_geo_0 = bootstrap(target_valid_geo_0, predictions_geo_0)
print("geo_0")
print("Средняя прибыль:", mean_geo_0)
print("Нижняя граница доверительного интервала", lower_geo_0)
print("Верхняя граница доверительного интервала", upper_geo_0)
print("Риск убытков {:.2%}".format(risk_geo_0))
      
risk_geo_1, lower_geo_1, upper_geo_1, mean_geo_1 = bootstrap(target_valid_geo_1, predictions_geo_1)
print("-----------------------------------------------------------")
print("geo_1")
print("Средняя прибыль:", mean_geo_1)
print("Нижняя граница доверительного интервала", lower_geo_1)
print("Верхняя граница доверительного интервала", upper_geo_1)
print("Риск убытков {:.2%}".format(risk_geo_1))
      
risk_geo_2, lower_geo_2, upper_geo_2, mean_geo_2 = bootstrap(target_valid_geo_2, predictions_geo_2)
print("-----------------------------------------------------------")
print("geo_2")
print("Средняя прибыль:", mean_geo_2)
print("Нижняя граница доверительного интервала", lower_geo_2)
print("Верхняя граница доверительного интервала", upper_geo_2)
print("Риск убытков {:.2%}".format(risk_geo_2))

geo_0
Средняя прибыль: 427847560.4625241
Нижняя граница доверительного интервала -97249829.56859529
Верхняя граница доверительного интервала 954215192.708815
Риск убытков 5.50%
-----------------------------------------------------------
geo_1
Средняя прибыль: 511362776.1976853
Нижняя граница доверительного интервала 98870649.90277725
Верхняя граница доверительного интервала 940720511.6508039
Риск убытков 0.90%
-----------------------------------------------------------
geo_2
Средняя прибыль: 402575607.5141956
Нижняя граница доверительного интервала -137162225.07193202
Верхняя граница доверительного интервала 929887528.0253205
Риск убытков 7.40%


# <a id='00'> Вывод </a> 
<a href='#head'>[head]</a>

Выходит наиболее подходящий регион "1", на этапе обучения и проверки модели именно в нем мы получили наименьший RMSE, предположив что там более равномерно распределен product, в итоге применения техники bootstrap риск убытков там оказался меньше всех, а средняя прибыль больше, несмотря на наименьший средний запас сырья.